# Virtual Try-On: Garment Styling

There are often many different ways a particular garment can be worn - sleeves rolled up or down, waist tucked or
untucked, etc. We refer to this as styling. Nova Canvas supports the following optional styling parameters:

- `longSleeveStyle` - Applies to long-sleeve tops. `"SLEEVE_UP" | "SLEEVE_DOWN"` (Note: only works if the sleeves in the source image are short)
- `tuckingStyle` - Applies to tops that fall below the waist. `"UNTUCKED" | "TUCKED"` (Note: only works if the top in the source image is tucked)
- `outerLayerStyle` - Applies to tops that can open in the front such as jackets. `"CLOSED" | "OPEN"`

_✏️ **NOTE:** An alternate way to control styling is to provide your own mask image which implies a particular styling of the garment._

In the following example, let's style the garment so that the sleeves are rolled up.

<div style="display: flex; gap: 2em; margin-right: 2em">
  <div style="text-align: center; flex: 1">
    <img src="../images/vto-images/vto_garment_styling_source.jpg">
    <strong><tt>sourceImage</tt></strong>
  </div>
  <div style="text-align: center; flex: 1">
    <img src="../images/vto-images/vto_garment_styling_reference.jpg">
    <strong><tt>referenceImage</tt></strong>
  </div>
   <div style="text-align: center; flex: 1">
    <img src="../images/vto-images/vto_garment_styling_output.png">
    <strong>Example output</strong>
  </div>
</div>


In [ ]:
import logging
from datetime import datetime
from random import randint
import base64
from amazon_image_gen import BedrockImageGenerator
import file_utils


logging.basicConfig(format="[%(levelname)s] %(message)s", level=logging.INFO)

# Edit these values to experiment with your own images.
source_image_path = "../images/vto-images/vto_garment_styling_source.jpg"
reference_image_path = "../images/vto-images/vto_garment_styling_reference.jpg"

# Load the source image from disk.
with open(source_image_path, "rb") as image_file:
    source_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")
# Load the reference image from disk.
with open(reference_image_path, "rb") as image_file:
    reference_image_base64 = base64.b64encode(image_file.read()).decode("utf-8")

inference_params = {
    "taskType": "VIRTUAL_TRY_ON",
    "virtualTryOnParams": {
        "sourceImage": source_image_base64,
        "referenceImage": reference_image_base64,
        "maskType": "GARMENT",
        "garmentBasedMask": {
            "garmentClass": "UPPER_BODY",
            # Add garment styling parameters
            "garmentStyling": {"longSleeveStyle": "SLEEVE_UP"},
        },
    },
    # The following is optional but provided here for you to experiment with.
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "quality": "standard",
        "cfgScale": 4.5,
        "seed": randint(0, 2147483646),
    },
}


# Define an output directory with a unique name.
generation_id = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
output_directory = f"output/{generation_id}"

# Create the generator.
generator = BedrockImageGenerator(output_directory=output_directory)

# Generate the image(s).
response = generator.generate_images(inference_params)

if "images" in response:
    # Save and display each image
    images = file_utils.save_base64_images(
        response["images"], output_directory, "image"
    )
    for image in images:
        display(image)